### The simple model, written in pytorch

In [5]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.datasets import imdb
from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 50
kernel_size = 3
epochs = 4

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


x_train = torch.LongTensor(sequence.pad_sequences(x_train, maxlen=maxlen))
x_test = torch.LongTensor(sequence.pad_sequences(x_test, maxlen=maxlen))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print (x_train[:10])
print (y_train[:10])

y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)

class Model(nn.Module):
    def __init__(self, embedding_dims, max_features, filters, kernel_size, maxlen):
        super(Model, self).__init__()

        self.embeddings = nn.Embedding(max_features, embedding_dims)
        self.conv1 = nn.Conv1d(embedding_dims, filters, kernel_size, padding=1)
        self.max_pooling = nn.MaxPool1d(maxlen)
        self.output_layer = nn.Linear(filters, 2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        #shape of input: [batch_size, maxlen]
        x = self.embeddings(x)
        #shape of x: [batch_size, maxlen, embedding_dim]
        x = x.permute((0,2,1))
        #shape of x: [batch_size, embedding_dim, maxlen]
        x = self.conv1(x)
        #shape of x: [batch_size, filters, maxlen]
        x = F.relu(x)
        #shape of x: [batch_size, filters, maxlen]
        x = self.max_pooling(x)
        #shape of x: [batch_size, filters, 1]
        x = x.squeeze()
        #shape of x: [batch_size, filters]
        x = self.output_layer(x)
        #shape of x: [batch_size, 2]
        x = self.softmax(x)
        #shape of x: [batch_size, 2]
        return x

model = Model(embedding_dims, max_features, filters, kernel_size, maxlen)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

#adapted from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(0, len(x_train), batch_size):

        inputs = x_train[i:i+batch_size]
        labels = y_train[i:i+batch_size]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        #Calculate gradients
        loss.backward()
        #Let optimizer update weights
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0 and i > 0:    # print every 100
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    #Let's calculate accuracy after an epoch
    answers = []
    for i in range(0, len(x_test), batch_size):
        xx = model(x_test[i:i+batch_size])
        #2 dimensional output to 1d class vector
        values, indices = torch.max(xx, dim=1)
        answers.extend((indices == torch.tensor(y_test[i:i+batch_size])).tolist())
    print ('acc', answers.count(True)/len(answers))

Loading data...
25000 train sequences
25000 test sequences
x_train shape: torch.Size([25000, 400])
x_test shape: torch.Size([25000, 400])
tensor([[  0,   0,   0,  ...,  19, 178,  32],
        [  0,   0,   0,  ...,  16, 145,  95],
        [  0,   0,   0,  ...,   7, 129, 113],
        ...,
        [595,  13, 258,  ...,  72,  33,  32],
        [  0,   0,   0,  ...,  28, 126, 110],
        [  0,   0,   0,  ...,   7,  43,  50]])
[1 0 0 1 0 0 1 0 1 0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[1,   161] loss: 0.409
[1,   321] loss: 0.352
[1,   481] loss: 0.345
[1,   641] loss: 0.346
[1,   801] loss: 0.348
[1,   961] loss: 0.339
[1,  1121] loss: 0.346
[1,  1281] loss: 0.345
[1,  1441] loss: 0.350
[1,  1601] loss: 0.336
[1,  1761] loss: 0.350
[1,  1921] loss: 0.348
[1,  2081] loss: 0.341
[1,  2241] loss: 0.336
[1,  2401] loss: 0.343
[1,  2561] loss: 0.340
[1,  2721] loss: 0.339
[1,  2881] loss: 0.340
[1,  3041] loss: 0.335
[1,  3201] loss: 0.336
[1,  3361] loss: 0.336
[1,  3521] loss: 0.331
[1,  3681] loss: 0.329
[1,  3841] loss: 0.331
[1,  4001] loss: 0.333
[1,  4161] loss: 0.326
[1,  4321] loss: 0.331
[1,  4481] loss: 0.326
[1,  4641] loss: 0.328
[1,  4801] loss: 0.333
[1,  4961] loss: 0.333
[1,  5121] loss: 0.323
[1,  5281] loss: 0.317
[1,  5441] loss: 0.323
[1,  5601] loss: 0.315
[1,  5761] loss: 0.322
[1,  5921] loss: 0.319
[1,  6081] loss: 0.323
[1,  6241] loss: 0.324
[1,  6401] loss: 0.311
[1,  6561] loss: 0.316
[1,  6721] loss: 0.307
[1,  6881] loss: 0.309
[1,  7041] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


acc 0.79588
[2,   161] loss: 0.288
[2,   321] loss: 0.256
[2,   481] loss: 0.254
[2,   641] loss: 0.262
[2,   801] loss: 0.248
[2,   961] loss: 0.264
[2,  1121] loss: 0.248
[2,  1281] loss: 0.262
[2,  1441] loss: 0.242
[2,  1601] loss: 0.245
[2,  1761] loss: 0.263
[2,  1921] loss: 0.248
[2,  2081] loss: 0.264
[2,  2241] loss: 0.265
[2,  2401] loss: 0.266
[2,  2561] loss: 0.262
[2,  2721] loss: 0.266
[2,  2881] loss: 0.259
[2,  3041] loss: 0.255
[2,  3201] loss: 0.240
[2,  3361] loss: 0.257
[2,  3521] loss: 0.237
[2,  3681] loss: 0.238
[2,  3841] loss: 0.252
[2,  4001] loss: 0.260
[2,  4161] loss: 0.247
[2,  4321] loss: 0.244
[2,  4481] loss: 0.244
[2,  4641] loss: 0.262
[2,  4801] loss: 0.250
[2,  4961] loss: 0.270
[2,  5121] loss: 0.241
[2,  5281] loss: 0.230
[2,  5441] loss: 0.241
[2,  5601] loss: 0.249
[2,  5761] loss: 0.249
[2,  5921] loss: 0.245
[2,  6081] loss: 0.253
[2,  6241] loss: 0.265
[2,  6401] loss: 0.233
[2,  6561] loss: 0.237
[2,  6721] loss: 0.243
[2,  6881] loss: 0.238